# Data Analyst Skills Analysis

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import datetime
import pandas as pd
import numpy as np
import missingno as msno
from string import punctuation as punct
import datetime as dt

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import re

pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [ ]:
jobs_main = pd.read_csv('gsearch_jobs.csv')
jobs = jobs_main.copy()

# Drop irrelevant columns
jobs.drop(columns=['Unnamed: 0', 'index',
                   'thumbnail', 'job_id', 'commute_time'], inplace=True)

# Drop duplicates
jobs.drop_duplicates(inplace=True)
jobs.reset_index()

jobs.head(10)

In [ ]:
jobs.info()

# Cleaning the Dataframe
The dataframe will be cleaned as follows:
1. We drop duplicates and irrelevant columns
2. We clean the 'via' column to 'via' from the records and update column name to listing_site
3. We handle null values in location & create a 'State' field from the location field.
4. We handle null values in schedule_type using the most frequent value, since this is a categorical feature.
5. We clean the description_token field to highlight top technical skills required for roles in the dataset.
6.

In [ ]:
msno.matrix(jobs)

In [ ]:
jobs.isnull().sum()

### Clean the 'via' column
Rename the column and using the series.apply() with a lambda function to remove 'via' from the records. The result is a series with the name of the sites where the jobs were listed.

In [ ]:
# Rename and clean the 'via' column
jobs.rename(columns={'via':'listing_site'}, inplace=True)
jobs['listing_site'] = jobs['listing_site'].apply(lambda x: x[4:])

jobs.head()

### Drop the extra salary columns
This is to let us conduct our own cleaning and analysis to determine what parameters 'salary' should be determined by/ displayed as.

In [ ]:
jobs.drop(columns=['salary_pay', 'salary_rate', 'salary_avg', 'salary_min',
                   'salary_max', 'salary_hourly', 'salary_yearly', 'salary_standardized'], inplace=True)

### Clean date_time columns
Convert the date_time column from object datatype to datetime datatype. The resulting datatype 'M8\[ns]' is a form of datetime datatype.

In [ ]:
jobs['date_time'] = pd.to_datetime(jobs['date_time'], errors='ignore')
jobs['date_time'].dtype

### Handle nulls in location
Use the description field to determine the location of the job. Where unable, use 'United States' as the most popular country option and the single option in search_location.

In [ ]:
jobs['location'].value_counts(dropna=False)

In [ ]:
jobs.loc[jobs['location'].isnull()]['description']

This record description shows that the location description is in Kansas City, MO. We'll update accordingly.

In [ ]:
jobs.loc[jobs['location'].isnull()]['description'].loc[652]

In [ ]:
jobs.at[652, 'location'] = 'Kansas City, MO'
jobs['location'].value_counts()

Using .loc[] and index, update the remaining null values with 'United States'

In [ ]:
jobs.loc[jobs['location'].isnull()].index

In [ ]:
jobs.loc[jobs.loc[jobs['location'].isnull()].index, 'location'] = 'United States'
jobs['location'].value_counts()

### Create a 'State' Column
Use the series.apply() method and a lambda function to separate state name from location where available and apply to a new field, 'state'. If no state exists, we'll apply the value in location to the state field.

In [ ]:
jobs['state'] = jobs['location'].apply(lambda x: x[-2:] if ',' in x else x)
jobs['state'] = jobs['state'].apply(lambda x: 'KS' if x == 'Kansas' else x)
jobs['state'] = jobs['state'].apply(lambda x: 'MO' if x == 'Missouri' else x)
jobs['state'] = jobs['state'].apply(lambda x: 'OK' if x == 'Oklahoma' else x)
jobs['state'].value_counts(dropna=False)

### Handle nulls in schedule_type
As a categorical column we'll use the mode to update the null values.

In [ ]:
jobs['schedule_type'].value_counts(normalize=True, dropna=False)

In [ ]:
# Handle nulls in schedule_type - using the max value
jobs.loc[jobs.loc[jobs['schedule_type'].isnull()].index, 'schedule_type'] = 'Full-time'
jobs['schedule_type'].value_counts(normalize=True, dropna=False)

### Clean Description Token

In [ ]:
jobs['description_tokens'].sample(10)

Convert this column to a list

In [ ]:
ts = jobs['description_tokens'].tolist()
ts[0:10]

Clean the list to remove brackets and pass cleaned list to a new variable.

In [ ]:
new_ts = list()
for i in ts:
    i = i.strip()
    if i == '':
        pass
    else:
        if '[' in i:
            i = i.replace('[', '')
        if ']' in i:
            i = i.replace(']', '')
        new_ts.append(i)

new_ts[0:10]

Convert cleaned list back to string using the .join() method and the ',' as the separator. Also remove trailing spaces. This allows us to concatenate all the records as a single string by the ',' value instead of being separated like in the original list values.

In [ ]:
ts_str = ','.join([str(elem) for elem in new_ts])
ts_str = ts_str.strip()
ts_str[:10]
# Result is a 10 character slice of the whole string

Create a copy of the string for further analysis so if we make any mistakes, we can start correcting from this point.

In [ ]:
ch_ts = ts_str
ch_ts[:10]

Convert string to list again using the separator ',' and selecting only unique values using the set() function. Also clean the values by removing empty string values. The result is list of unique values

In [ ]:
ch_ts = ch_ts.strip().split(',')
ch_ts = list(set(ch_ts))
ch_ts.remove('')
ch_ts[:10]

Clean the list to remove trailing spaces and quotation marks. Sort resulting list in alphabetical order. The result is now a clear list of tech skills in data analysis.

In [ ]:
tech_skills = list()
for i in ch_ts:
    a = i.strip()
    tech_skills.append(a[1:-1])

tech_skills = sorted(list(set(tech_skills)))

print(tech_skills, len(tech_skills))

Create a copy of the jobs dataframe, so that if any errors are made, we can correct them from this point.

Pass the values in the tech_skills list as columns in the jobs_copy dataframe.

In [ ]:
jobs_copy = jobs.copy()
for i in tech_skills:
    jobs_copy[i] = 0

jobs_copy.head()

Create a function that assigns a 1 into the relevant record / field if they exist as skills in the description_tokens column.

In [ ]:
def update_tech_skills(column, tech_list, df):
    for index, value in column.items():
        for i in tech_skills:
            if i in value:
                df.at[index, i] = 1

    return df

Apply function to dataframe and pass the result back to the dataframe variable.

In [ ]:
jobs_copy = update_tech_skills(jobs_copy['description_tokens'], tech_skills, jobs_copy)
jobs_copy[tech_skills].sample(20)

In [ ]:
for i in tech_skills:
    print(jobs_copy[i].value_counts(dropna=False))
    print('-'*8)

### Drop tech skills columns with Values under 100
Given that there are 98 unique tech skills, this makes our dataframe too wide. We'll reduce the skills to those that appear more than 100 times in the dataset.

Let's drop the tech skills columns whose sums are under 100. The resulting dataframe has only 17 tech skills. This is better.

In [ ]:
for i in tech_skills:
    if jobs_copy[i].sum() < 100:
        jobs_copy.drop(columns=i, inplace=True)

In [ ]:
for i in tech_skills:
    if i in jobs_copy.columns:
        print(jobs_copy[i].value_counts(dropna=False))
        print('-'*8)

In [ ]:
jobs_copy.sample(10)

### Drop irrelevant columns
Also rename state column to location.

In [ ]:
jobs_copy.drop(columns=['location', 'posted_at', 'search_term', 'search_location', 'description_tokens'], inplace=True)
jobs.rename(columns={'state':'location'}, inplace=True)
jobs_copy.sample(5)

### Clean Salary Columns

In [ ]:
jobs_copy['salary'].value_counts(dropna=False)

Using series.apply() method and a lambda function, extract rate (hour, month, year) from the salary field and pass the result to a new field - salary_rate. This is relevant to help us know which salaries to convert to an hourly rate and by which conversion rate.

In [ ]:
jobs_copy['salary_rate'] = jobs_copy['salary'].apply(lambda x: x[-5:] if type(x) == str else x)
jobs_copy['salary_rate'] = jobs_copy['salary_rate'].apply(lambda x: x.strip() if type(x) == str else x)
jobs_copy['salary_rate'] = jobs_copy['salary_rate'].apply(lambda x: 'N/A' if type(x) != str else x)
jobs_copy['salary_rate'].value_counts(dropna=False)

Remove the rate section from the salary records and convert nan values to 0.

In [ ]:
jobs_copy['salary'] = jobs_copy['salary'].apply(lambda x: x[:-7].strip() if type(x) == str else 0)
jobs_copy['salary'].value_counts(dropna=False)

Convert all values in the salary field to string datatype.
For records with '-' in their values, use that as a split separator and extract the resulting two values as min and max values. For those without, use their single values as min and max values.

The resulting two fields depict the salary range as minimum and maximum values.

In [ ]:
jobs_copy['salary'] = jobs_copy['salary'].astype(str)

jobs_copy['salary_min'] = jobs_copy['salary'].apply(lambda x: x.split('–')[0] if len(x) > 3 else x)
jobs_copy['salary_max'] = jobs_copy['salary'].apply(lambda x: x.split('–')[-1] if len(x) > 3 else x)

Remove ',' in the records and replace 'K' with '000' to reflect thousands.

In [ ]:
jobs_copy['salary_min'] = jobs_copy['salary_min'].str.replace(',','').str.replace('K','000', flags=re.I)
jobs_copy['salary_min'].value_counts()

In [ ]:
jobs_copy['salary_max'] = jobs_copy['salary_max'].str.replace(',','').str.replace('K','000', flags=re.I)
jobs_copy['salary_max'].value_counts()

In [ ]:
jobs_copy.loc[jobs_copy['salary_rate'] == 'year'][['salary_min', 'salary_max', 'salary_rate']]

Convert the salary range values to float. For year_rate salary values under 200, add '00' to reflect true values.

In [ ]:
jobs_copy[['salary_min', 'salary_max']] = jobs_copy[['salary_min', 'salary_max']].astype(float)

In [ ]:
salary = jobs_copy[['salary_min', 'salary_max']].astype(float)
salary['salary_rate'] = jobs_copy['salary_rate']
salary.head()

In [ ]:
sal_str = salary.loc[(salary['salary_min'] < 200) & (salary['salary_rate'] == 'year')]['salary_min'].astype(str).str.replace('.','')
sal_str = sal_str.apply(lambda x: x + '00')
sal_str

In [ ]:
jobs_copy.loc[sal_str.index, 'salary_min'] = sal_str.astype(float)

In [ ]:
salary.loc[(salary['salary_max'] < 200) & (salary['salary_rate'] == 'year')][['salary_min', 'salary_max', 'salary_rate']]

In [ ]:
sal_str2 = salary.loc[(salary['salary_max'] < 200) & (salary['salary_rate'] == 'year')]['salary_max'].astype(str).str.replace('.','')
sal_str2 = sal_str2.apply(lambda x: x + '00')
sal_str2

In [ ]:
jobs_copy.loc[sal_str2.index, 'salary_max'] = sal_str2.astype(float)

In [ ]:
jobs_copy.loc[jobs_copy['salary_rate'] == 'year'][['salary_min', 'salary_max', 'salary_rate']]

For year salary rate, divide values by 2080 (working hours in a year) where the

In [ ]:
yr_sal_min = jobs_copy.loc[jobs_copy['salary_min'] > 200]['salary_min']
yr_sal_max = jobs_copy.loc[jobs_copy['salary_max'] > 10000]['salary_max']

In [ ]:
len(yr_sal_max)

In [ ]:
yr_sal_max

In [ ]:
yr_sal_max = round(yr_sal_max / 2080, 2)
yr_sal_min = round(yr_sal_min / 2080, 2)

yr_sal_max

In [ ]:
jobs_copy.loc[yr_sal_min.index, 'salary_min'] = yr_sal_min
jobs_copy.loc[yr_sal_max.index, 'salary_max'] = yr_sal_max

In [ ]:
jobs_copy.loc[jobs_copy.loc[jobs_copy['salary_rate'] == 'month'].index, 'salary_min'] = round(5400/195, 2)

jobs_copy.loc[jobs_copy.loc[jobs_copy['salary_rate'] == 'month'].index, 'salary_max'] = round(6000/195, 2)

In [ ]:
jobs_copy.loc[jobs_copy['salary_rate'] == 'month'][['salary_min', 'salary_max', 'salary_rate']]

Drop irrelevant columns.

In [ ]:
jobs_copy.drop(columns=['salary', 'salary_rate', ], inplace=True)
jobs_copy.rename(columns={'salary_min':'salary_min_hr', 'salary_max':'salary_max_hr'}, inplace=True)

jobs_copy.sample(15)

# Clean the Extensions column
We're trying to find out if there's salary information in the extensions field where there is none in the salary fields so we can update accordingly.

Let's make a copy of the dataset so if we make any errors we can correct them from this point.

In [ ]:
jobs_2 = jobs_copy.copy()

In [ ]:
ext_desc = jobs_2.loc[(jobs_2['salary_min_hr'] == 0) & (jobs_2['salary_max_hr'] == 0)][['extensions', 'description']]

ext_desc['extensions'].head()

In [ ]:
indexes = list()
for index, value in ext_desc['extensions'].items():
    for item in value:
        if 'per year' in value:
            indexes.append(index)
        elif 'per hour' in value:
            indexes.append(index)

indexes

We will be dropping the extensions field as it contains no relevant salary information.

In [ ]:
jobs_2.drop(columns=['extensions'], inplace=True)
jobs_2.head()

In [ ]:
ext_desc.drop(columns=['extensions'], inplace=True)
ext_desc.head()

In [ ]:
indexes = list()
for index, value in ext_desc.items():
    for item in value:
        if 'year' in value:
            indexes.append(index)
        elif 'hour' in value:
            indexes.append(index)

indexes

We'll be dropping the description field as well as it contains no relevant salary information.

In [ ]:
jobs_2.drop(columns=['description'], inplace=True)
jobs_2.head()

### Option 2

Sample Cleaning code for this project from Kaggle user [Julia Nesterenko](https://www.kaggle.com/code/nesterenkojul/data-analyst-skill-worth)

In [ ]:
df = jobs_main.copy()
df = df.drop(columns=["Unnamed: 0", "index"])

In [ ]:
# Format lists
remove_punct = lambda x: x.translate(str.maketrans('', '', punct)).split()
df.extensions = df.extensions.apply(remove_punct)
df.description_tokens = df.description_tokens.apply(remove_punct)
df.description = df.description.apply(remove_punct)
df[['description', 'extensions', 'description_tokens']].sample(10)

Compare with original.

In [ ]:
jobs_main[['description', 'extensions', 'description_tokens']].sample(10)

In [ ]:
# Format dates
df.date_time = pd.to_datetime(df.date_time)
df.insert(14, "time_ago", df.date_time.apply(lambda x: dt.datetime.today() - x))
df.sample(5)

Step 1. Define Top 10 most popular (most frequently occurring) skills.
Step 2. Compute average yearly salaries for job postings that require those skills.

In [ ]:
top = 10
skills = []
# add all skills to a list
for val in df.description_tokens.values:
    skills.extend(val)

# Get list of unique skills and their frequency in tuples
skills, counts = np.unique(skills, return_counts=True)

# Sort above list by frequency. Limit skills list to top N frequency
top_skill_count = sorted(zip(list(skills), list(counts)), key=lambda x: -x[1])[:top]

# Separate skills from frequency counts
top_skills = list(map(lambda x: x[0], top_skill_count))
top_counts = list(map(lambda x: x[1], top_skill_count))

# Get average annual standardized salary for all listings with skill in their description_tokens field.
salaries = []
for skill in top_skills:
    salaries.append(df[df.description_tokens.apply(lambda x: skill in x)].salary_standardized.mean())

# Map results to a df and sort by average salary
top_skills_df = pd.DataFrame({"skill": list(top_skills), "number_of_postings": top_counts,"avg_yearly_salary": map(round, salaries)})
top_skills_df = top_skills_df.sort_values("avg_yearly_salary", ascending=False)
top_skills_df

## Exploratory Data Analysis